### Não esqueça de entrar na venv específica do lazy predict, obrigatório dado os problemas de compatibilidade do lazy predict com imbalance learn e scikit-learn

python3 -m venv venv

pip install -r src/lazypredict_requirements.txt

. venv/bin/activate

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from lazypredict.Supervised import LazyClassifier

ModuleNotFoundError: No module named 'lazypredict.Supervised'

In [11]:
def correcting_nan(data):
  data = data.replace("na", np.nan)
  data = data.replace("neg", 0)
  data = data.replace("pos", 1)
  data = data.apply(pd.to_numeric)
  return data

def remove_nan_rows_and_columns(data):
  column_thresh = data.shape[0]*NAN_COLUMN_THRESHOLD
  data = data.dropna(thresh=column_thresh, axis = 1)
  row_thresh = data.shape[1]*NAN_ROW_THRESHOLD
  data = data.dropna(thresh=row_thresh, axis = 0)
  return data

def fill_na_with_median(data):
  data = data.fillna(data.median())
  return data

def calculating_corr_matrix(data):
  corr = data.corr()
  sns.heatmap(corr)
  return data, corr

def deleting_high_correlation_columns(data, corr):
  columns = np.full((corr.shape[0],), True, dtype=bool)
  for i in range(corr.shape[0]):
      for j in range(i+1, corr.shape[0]):
          if corr.iloc[i,j] >= CORR_THRESHOLD:
              if columns[j]:
                  columns[j] = False
  selected_columns = data.columns[columns]
  data = data[selected_columns]
  return data, selected_columns

def separate_target_and_features(data):
  target = data[label]
  features = data.loc[:, data.columns != label]
  return target, features

def train_test_for_this_dataset(data):
    train = data.loc[data['2020'] == 0].loc[:, data.columns != flag_column]
    test = data.loc[data['2020'] == 1].loc[:, data.columns != flag_column]
    x_train = train.loc[:, train.columns != label]
    x_test = test.loc[:, test.columns != label]
    y_train = train[label]
    y_test = test[label]
    return x_train, x_test, y_train, y_test

In [ ]:
## preprocessing pipeline:

data = correcting_nan(data)
data = remove_nan_rows_and_columns(data)
data = fill_na_with_median(data)
data, corr = calculating_corr_matrix(data)
data, _ = deleting_high_correlation_columns(data, corr)

In [ ]:
from sklearn.preprocessing import PowerTransformer

def yeo_johnson(x):
    power = PowerTransformer(method='yeo-johnson', standardize=True)
    x = pd.DataFrame(power.fit_transform(x), columns = x.columns)
    return x

#removing bad columns
list_of_columns = ['cd_000', 'as_000', 'au_000', 'ch_000']

def removing_bad_columns(x, list_of_columns):
    x.drop(list_of_columns, inplace=True, axis=1)
    return x



In [ ]:
cls= LazyClassifier(ignore_warnings=False, custom_metric=None)
models, predictions = cls.fit(x_train, x_test, y_train, y_test)

In [ ]:
cls= LazyClassifier(ignore_warnings=False, custom_metric=None)
df_dict={}

for i in range(1,30):
    sample = data.sample(frac=0.10)
    x_train, x_test, y_train, y_test = train_test_for_this_dataset(sample)
    
    x_train = yeo_johnson(x_train)
    x_test = yeo_johnson(x_test)
    x_train = removing_bad_columns(x_train, list_of_columns)
    x_test = removing_bad_columns(x_test, list_of_columns)
    
    # transform the dataset
    oversample = SMOTE()
    x_train, y_train = oversample.fit_resample(x_train, y_train)

    
    models, predictions = cls.fit(x_train, x_test, y_train, y_test)
    df_dict[i] = models
    print('next run............')
    print(' ')
    print(' ')

In [ ]:
df_concat = pd.concat((df_dict)).reset_index()

In [ ]:
by_row_index = df_concat.groupby(df_concat['Model'])
df_means = by_row_index.mean()

In [ ]:
df_means.sort_values(by=['Balanced Accuracy'], ascending=False)